In [ ]:
# Human-level control through deep reinforcement learning

from torch import nn
import torch
import gym
from collections import deque
import itertools
import numpy as np
import random

In [ ]:
GAMMA = 0.99                                                                    # decaying rate
BATCH_SIZE = 32                                                                 # How many transitions we are going to sample from the replay buffer when we are computing our gradients
BUFFER_SIZE = 50000                                                             # Maximum number of transition we are going to store
MIN_REPLAY_SIZE = 1000                                                          # How many transition we want in the replay buffer before we start computing gradients
EPSILON_START = 1.0
EPSILON_END = 0.02
EPSILON_DECAY = 10000
TARGET_UPDATE_FREQ = 1000

In [ ]:
class Network(nn.Module):
    def __init__(self, env):
        super().__init__()

        in_features = int(np.prod(env.observation_space.shape))

        self.net = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.Tanh(),
            nn.Linear(64, env.action_space.n)
        )

    def forward(self, x):
        return self.net(x)

    def act(self, state):

        state_t = torch.as_tensor(state, dtype=torch.float32)

        q_values = self.forward(state_t.unsqueeze(0))                           # 'q_values' outputs two values (left or right)

        max_q_index = torch.argmax(q_values, dim=1)[0]                          # find an index that corresponds to the maximum value  

        action = max_q_index.detach().item()                                    # 0 or 1

        return action                                                           # 0 or 1

In [ ]:
env = gym.make('CartPole-v1')
replay_buffer = deque(maxlen=BUFFER_SIZE)
reward_buffer = deque([0.0], maxlen=100)
episode_reward = 0.0

online_net = Network(env)
target_net = Network(env)

target_net.load_state_dict(online_net.state_dict())                             # we set the target net parameters equal to the online network parameters

optimizer = torch.optim.Adam(online_net.parameters(), lr=5e-4)

In [ ]:
# Initialize Replay Buffer
# 최소 MIN_REPLAY_SIZE 만큼의 transition을 buffer에 넣고 시작한다. 
state = env.reset()

for _ in range(MIN_REPLAY_SIZE):

    action = env.action_space.sample()
    new_state, reward, done, _ = env.step(action)
    transition = (state, action, reward, done, new_state)
    replay_buffer.append(transition)
    state = new_state

    if done:                                                                    # 게임이 끝나게 되면 다시 리셋해주고 버퍼를 쌓아준다. 
        state = env.reset()

In [ ]:
# Main Training Loop

state = env.reset()                                                             # state의 예시: [-0.01713841 -0.00705756 -0.04146662 -0.04927411]

for step in itertools.count():                                                  # step starts from 0 and increases by 1 until it meets a break condition. This is same as 'While True' loop

    epsilon = np.interp(step, [0, EPSILON_DECAY], [EPSILON_START, EPSILON_END]) # epsilon value는 EPSILON_START에서 시작해서 EPSILON_END까지 step이 흘러갈수록 점점 더 감소한다. 

    random_sample = random.random()

    if random_sample <= epsilon:                                                # random_sample의 값이 epsilon보다 작으면, random한 action을 취하고, 그렇지 않다면 online_net에 현재 state를 넣어 가장 좋은 act를 가져온다. 
        action = env.action_space.sample()
    else:
        action = online_net.act(state)

    new_state, reward, done, _ = env.step(action)
    transition = (state, action, reward, done, new_state)
    replay_buffer.append(transition)                                            # step 1번에 transition 1번이 append된다. 
    state = new_state

    episode_reward  = episode_reward + reward

    if done:                                                                    # 게임이 끝나면(막대기가 넘어지면), done값이 True가 된다. 
        state = env.reset()                                                     # 끝난다면 env를 reset해주고, 
        reward_buffer.append(episode_reward)                                    # reward_buffer에 episode_reward를 append해준다. 
        episode_reward = 0.0

    # # -------------------------- TEST --------------------------
    # # After solved, watch it play
    # if len(reward_buffer) == 100:
    #     if np.mean(reward_buffer) >= 195:
    #         while True:
    #             action = online_net.act(state)

    #             action, _, done, _ = env.step(action)
    #             env.render()
    #             if done:
    #                 env.reset()
    # # -------------------------- TEST --------------------------

    # Start Gradient Step
    transitions = random.sample(replay_buffer, BATCH_SIZE)                      # replay_buffer에서 batch_size만큼의 sample을 가져온다. 

    states     = np.asarray([t[0] for t in transitions])                        # len(states) == 32
    actions    = np.asarray([t[1] for t in transitions])                        # len(actions) == 32
    rewards    = np.asarray([t[2] for t in transitions])                        # len(rewards) == 32    
    dones      = np.asarray([t[3] for t in transitions])                        # len(dones) == 32    
    new_states = np.asarray([t[4] for t in transitions])                        # len(new_states) == 32

    # print("states -->", states[0])
    # print("actions -->", actions[0])
    # print("rewards -->", rewards[0])
    # print("dones -->", dones[0])
    # print("new_states -->", new_states[0])
    
    states_t     = torch.as_tensor(states, dtype=torch.float32)
    actions_t    = torch.as_tensor(actions, dtype=torch.int64).unsqueeze(-1)
    rewards_t    = torch.as_tensor(rewards, dtype=torch.float32).unsqueeze(-1)
    dones_t      = torch.as_tensor(dones, dtype=torch.float32).unsqueeze(-1)
    new_states_t = torch.as_tensor(new_states, dtype=torch.float32)

    # Compute Targets
    target_q_values = target_net.forward(new_states_t)
    max_target_q_values = target_q_values.max(dim=1, keepdim=True)[0]           
    targets = rewards_t + GAMMA * (1 - dones_t) * max_target_q_values           # 32 targets are computed. See Algorithm 1 of Human-level control through deep reinforcement learning (Nature14236). 

    # Compute Loss
    q_values = online_net.forward(states_t)                             
    action_q_values = torch.gather(input=q_values, dim=1, index=actions_t)      # 32 action values are computed 
    loss = nn.functional.smooth_l1_loss(action_q_values, targets)

    # Gradient Descent
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Update Target Network
    if step % TARGET_UPDATE_FREQ == 0:
        target_net.load_state_dict(online_net.state_dict())                     # we set the target net parameters equal to the online network parameters

    # Logging
    if step % 1000 == 0:
        print()
        print('Step', step)
        print('Avg Reward', np.mean(reward_buffer))                             # maximum length of reward_buffer is 100. Therefore, np.mean(reward_buffer) averages lastest 100 rewards


Step 0
Avg Reward 0.0

Step 1000
Avg Reward 22.613636363636363

Step 2000
Avg Reward 23.22093023255814

Step 3000
Avg Reward 22.02

Step 4000
Avg Reward 23.32

Step 5000
Avg Reward 24.34

Step 6000
Avg Reward 28.43

Step 7000
Avg Reward 33.95

Step 8000
Avg Reward 41.38

Step 9000
Avg Reward 48.58

Step 10000
Avg Reward 57.37

Step 11000
Avg Reward 66.22

Step 12000
Avg Reward 75.51

Step 13000
Avg Reward 84.27

Step 14000
Avg Reward 92.15

Step 15000
Avg Reward 100.31

Step 16000
Avg Reward 109.48

Step 17000
Avg Reward 116.96

Step 18000
Avg Reward 124.8

Step 19000
Avg Reward 132.97


KeyboardInterrupt: ignored